In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
listed = drive.ListFile({'q': "title contains '.pkl' and 'root' in parents"}).GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))
    

title data_75_fasttext.pkl, id 1OcIBhCyhsqbaH1Mr7zu1zjDr_tUQKEii
title refined_giacomo_75.pkl, id 1fAFqsIYNQWKBcV8pGzIPqKDM6Gql95v3
title base_df.pkl, id 1cqUZDd2Wl--LdnzWpBuRn_dJt8PFm_LD


In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '1cqUZDd2Wl--LdnzWpBuRn_dJt8PFm_LD'
request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
f = pickle.load(downloaded)
base_df=f

In [0]:
base_df.head()

,__label__,tweets
0,__label__Right,RT @luigivanti: Il problema non si risolve rin...
1,__label__Left,@Ire_Lica @salvinimi @DarioSegreto_ @maddale...
2,__label__Unknown,Un #cinema per un popolo che manca. @TheIris...
3,__label__Right,b'RT @petergomezblog: Da gioved\xc3\xac 5 dice...
4,__label__Left,RT @RaiStoria: La Gran Bretagna è il paese p...


In [0]:
!pip install fasttext -f https://github.com/facebookresearch/fastText.git


Looking in links: https://github.com/facebookresearch/fastText.git


In [0]:
import pandas as pd 
import fasttext

In [0]:
base_df.head()
refined_df = base_df
control_df = base_df.copy()

In [0]:
#remove html tags
from bs4 import BeautifulSoup

refined_df["tweets"] = base_df["tweets"].apply(lambda s: BeautifulSoup(s).get_text())

In [0]:
#remove hashtags and mentions
# tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
import re
refined_df["tweets"] = base_df["tweets"].apply(lambda s: ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", s).split()))


In [0]:
#remove urls
refined_df["tweets"] = base_df["tweets"].apply(lambda s: ' '.join(re.sub("(\w+:\/\/\S+)", " ", s).split()))

In [0]:
#remove punctuations
refined_df["tweets"] = base_df["tweets"].apply(lambda s: ' '.join(re.sub("[\.\,\!\?\:\;\-\=\'\...\"]", " ", s).split()))

In [0]:
refined_df["tweets"] = base_df["tweets"].apply(lambda s: s.lower())

In [0]:
#remove emoji since the package translates to english, too lazy to make custom
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

refined_df["tweets"] = base_df["tweets"].apply(lambda s: deEmojify(s))

In [0]:
#remmove stopwords
stop_words = ['a',
       'b',
       'c',
       'd',
       'e',
       'f',
       'g',
       'h',
       'i',
       'l',
       'm',
       'n',
       'o',
       'p',
       'q',
       'r',
       's',
       't',
       'u',
       'v',
       'z',
       'y',
       'w',
       'x',
       'j',
       'k',
       'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche', 'Molisie', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 'Trentino-Alto Adige', 'Umbria', 'Valle d Aosta', 'Veneto', 
       '2030',
       '2029',
       '2028',
       '2027',
       '2026',
       '2025',
       '2024',
       '2023',
       '2022',
       '2021',
       '2020',
       '2019',
       '2018',
       '2017',
       '2016',
       '2015',
       '2014',
       '2013',
       '2012',
       '2011',
       '2010',
       '2009',
       '2008',
       '2007',
       '2006',
       '2005',
       '2004',
       '2003',
       '2002',
       '2001',
       '2000',
       '1999',
       '1998',
       '1997',
       '1996',
       '1995',
       '1994',
       '1993',
       '1992',
       '1991',
       '1990',
       '1989',
       '1988',
       '1987',
       '1986',
       '1985',
       '1984',
       '1983',
       '1982',
       '1981',
       '1980',    
  'abbia',
 'abbiamo',
 'abbiano',
 'abbiate',
 'ad',
 'adesso',
 'agl',
 'agli',
 'ai',
 'al',
 'all',
 'alla',
 'alle',
 'allo',
 'allora',
 'altre',
 'altri',
 'altro',
 'anche',
 'ancora',
 'avemmo',
 'avendo',
 'avere',
 'avesse',
 'avessero',
 'avessi',
 'avessimo',
 'aveste',
 'avesti',
 'avete',
 'aveva',
 'avevamo',
 'avevano',
 'avevate',
 'avevi',
 'avevo',
 'avrai',
 'avranno',
 'avrebbe',
 'avrebbero',
 'avrei',
 'avremmo',
 'avremo',
 'avreste',
 'avresti',
 'avrete',
 'avrà',
 'avrò',
 'avuta',
 'avute',
 'avuti',
 'avuto',
 'c',
 'che',
 'chi',
 'ci',
 'coi',
 'col',
 'come',
 'con',
 'contro',
 'cui',
 'da',
 'dagl',
 'dagli',
 'dai',
 'dal',
 'dall',
 'dalla',
 'dalle',
 'dallo',
 'degl',
 'degli',
 'dei',
 'del',
 'dell',
 'della',
 'delle',
 'dello',
 'dentro',
 'di',
 'dov',
 'dove',
 'e',
 'ebbe',
 'ebbero',
 'ebbi',
 'ecco',
 'ed',
 'era',
 'erano',
 'eravamo',
 'eravate',
 'eri',
 'ero',
 'essendo',
 'faccia',
 'facciamo',
 'facciano',
 'facciate',
 'faccio',
 'facemmo',
 'facendo',
 'facesse',
 'facessero',
 'facessi',
 'facessimo',
 'faceste',
 'facesti',
 'faceva',
 'facevamo',
 'facevano',
 'facevate',
 'facevi',
 'facevo',
 'fai',
 'fanno',
 'farai',
 'faranno',
 'fare',
 'farebbe',
 'farebbero',
 'farei',
 'faremmo',
 'faremo',
 'fareste',
 'faresti',
 'farete',
 'farà',
 'farò',
 'fece',
 'fecero',
 'feci',
 'fino',
 'fosse',
 'fossero',
 'fossi',
 'fossimo',
 'foste',
 'fosti',
 'fra',
 'fu',
 'fui',
 'fummo',
 'furono',
 'giù',
 'gli',
 'ha',
 'hai',
 'hanno',
 'ho',
 'i',
 'il',
 'in',
 'io',
 'l',
 'la',
 'le',
 'lei',
 'li',
 'lo',
 'loro',
 'lui',
 'ma',
 'me',
 'mi',
 'mia',
 'mie',
 'miei',
 'mio',
 'ne',
 'negl',
 'negli',
 'nei',
 'nel',
 'nell',
 'nella',
 'nelle',
 'nello',
 'no',
 'noi',
 'non',
 'nostra',
 'nostre',
 'nostri',
 'nostro',
 'o',
 'per',
 'perché',
 'però',
 'più',
 'pochi',
 'poco',
 'qua',
 'quale',
 'quanta',
 'quante',
 'quanti',
 'quanto',
 'quasi',
 'quella',
 'quelle',
 'quelli',
 'quello',
 'questa',
 'queste',
 'questi',
 'questo',
 'qui',
 'quindi',
 'sarai',
 'saranno',
 'sarebbe',
 'sarebbero',
 'sarei',
 'saremmo',
 'saremo',
 'sareste',
 'saresti',
 'sarete',
 'sarà',
 'sarò',
 'se',
 'sei',
 'senza',
 'si',
 'sia',
 'siamo',
 'siano',
 'siate',
 'siete',
 'sono',
 'sopra',
 'sotto',
 'sta',
 'stai',
 'stando',
 'stanno',
 'starai',
 'staranno',
 'stare',
 'starebbe',
 'starebbero',
 'starei',
 'staremmo',
 'staremo',
 'stareste',
 'staresti',
 'starete',
 'starà',
 'starò',
 'stava',
 'stavamo',
 'stavano',
 'stavate',
 'stavi',
 'stavo',
 'stemmo',
 'stesse',
 'stessero',
 'stessi',
 'stessimo',
 'stesso',
 'steste',
 'stesti',
 'stette',
 'stettero',
 'stetti',
 'stia',
 'stiamo',
 'stiano',
 'stiate',
 'sto',
 'su',
 'sua',
 'sue',
 'sugl',
 'sugli',
 'sui',
 'sul',
 'sull',
 'sulla',
 'sulle',
 'sullo',
 'suo',
 'suoi',
 'te',
 'ti',
 'tra',
 'tu',
 'tua',
 'tue',
 'tuo',
 'tuoi',
 'tutti',
 'tutto',
 'un',
 'una',
 'uno',
 'vai',
 'vi',
 'voi',
 'vostra',
 'vostre',
 'vostri',
 'vostro',
 'è',
'doppo',
'dopo',
    'rt',
    'b',
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]


# Code to remove noisy words from a text

def _remove_noise(input_text):
    words = input_text.lower()
    words = words.split()
    noise_free_words = [word for word in words if word not in stop_words] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [0]:
#remmove stopwords
refined_df["tweets"] = base_df["tweets"].apply(lambda s: _remove_noise(s))

In [0]:
#remove unknowns profile
refined_df=refined_df[refined_df.loc[:,'__label__']!="__label__Unknown"]

In [0]:
col_names = ['__label__',"tweets"]
train_df = pd.DataFrame(columns = col_names)
test_df = pd.DataFrame(columns = col_names)
train_df = refined_df.iloc[:1760,:]
test_df = refined_df.iloc[1760:,:]


In [0]:
import csv
train_df['tweets']= train_df['tweets'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
test_df['tweets']= test_df['tweets'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
train_df.to_csv('train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ") 
test_df.to_csv('test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ") 

In [0]:
train_df['tweets']

In [0]:
import fasttext
model = fasttext.train_supervised(input="train.txt",epoch=50)
model.save_model("model_twitter.bin")

In [0]:
model.predict("dobbiamo favorire le unioni civile siamo nel 2020")

(('__label__Right',), array([0.9999429]))

In [0]:
model.test("test.txt")

(440, 0.8159090909090909, 0.8159090909090909)